In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
def generate_dataframes():
    
    directory_path = '../raw_data/'
    file_names = []

    for i in range(1,26):
        if i < 10 :
            file_name = f'A0{i}.csv'
        else :
            file_name = f'A{i}.csv'
        file_names.append(file_name)

    dataframes = {}
    for file in file_names:
        df = pd.read_csv(directory_path+file)
        dataframes[file] = df
    return dataframes

def add_timestamps():
    results = {}
    for file in dataframes.keys():
        df = dataframes[file]
        df['Fecha'] = pd.to_datetime(df['Fecha'])
        df.set_index('Fecha',inplace=True)
        ref_date_range = pd.date_range(start='1/05/2019', end='30/09/2021',freq='10T')
        ref_df = pd.DataFrame(index=ref_date_range)
        clean_data = df.reindex(ref_df.index)
        new_df = pd.merge(ref_df,clean_data,left_index=True, right_index=True,how='outer')
        results[file] = new_df
    results['A02.csv'].drop(columns='Unnamed: 7',inplace= True)
    return results


In [9]:
dataframes = generate_dataframes()

results = add_timestamps()


/var/folders/ng/p31bnd8n1239g9smsknq3k600000gn/T/ipykernel_14448/3426815231.py:3: UserWarning: Parsing '30/09/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  results = add_timestamps()
/var/folders/ng/p31bnd8n1239g9smsknq3k600000gn/T/ipykernel_14448/3426815231.py:3: UserWarning: Parsing '30/09/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  results = add_timestamps()
/var/folders/ng/p31bnd8n1239g9smsknq3k600000gn/T/ipykernel_14448/3426815231.py:3: UserWarning: Parsing '30/09/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  results = add_timestamps()
/var/folders/ng/p31bnd8n1239g9smsknq3k600000gn/T/ipykernel_14448/3426815231.py:3: UserWarning: Parsing '30/09/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  results = add_timestamps()
/var/folders/ng/p31bnd8n

KeyboardInterrupt: 

- Subsample function : iterate over 25 files 
- Generate X number of subdatasets 
- Check how many contain unaccepatable levels (X) of missing values 
- Split en X et en Y 

In [297]:
def subsample_sequence(results, day_length, number_of_subsamples, acceptable_level_of_missing_values = 0.1):
    """
    Given the initial dictionnary of dataframes `results`, return a list of dataframes of length `length`.
    """
    length = day_length*6*24
    last_possible = df.shape[0] - length
    random_start = np.random.randint(0, last_possible)
    
    subsamples = []
    
    while len(subsamples) < number_of_subsamples :
        random_file_number = np.random.randint(1, 26)
        
        if random_file_number < 10:
            file_name = f'A0{random_file_number}.csv'
        else :
            file_name = f'A{random_file_number}.csv'
        df_sample = results[file_name][random_start: random_start+int(length)]
        
        if (df_sample.isna().sum()/len(df_sample))[0] < acceptable_level_of_missing_values :
            subsamples.append(df_sample)
            # $CHALLENGIFY_END
    return subsamples

In [299]:
def split_subsample_sequence(results, day_length, number_of_sumbsamples, acceptable_level_of_missing_values = 0.1):
    '''Create one single random (X,y) pair'''
    # $CHALLENGIFY_BEGIN
    length = day_length*6*24
    subsamples = subsample_sequence(results, day_length, number_of_sumbsamples, acceptable_level_of_missing_values = 0.1)
    Y = []
    X = []
    for sample in subsamples:
        y_sample = sample[['Media de Potencia Activa 10M\n(kW)']].iloc[sample.shape[0]-72:]
        Y.append(y_sample)

        x_sample = sample[0:int(length) -72]
        X.append(x_sample)
    # $CHALLENGIFY_END
    return np.array(X), np.array(Y)